<font face = "Arial" size = 6> Model </font>

In [3]:
import string
import numpy as np
from collections import Counter
from math import log, exp
import math

class NaiveBayesClassifier:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing  # Laplace Smoothing
        self.class_probs = {}  # P(C)
        self.word_probs_by_class = {}  # P(word | class)
        self.vocab = set()  # Từ điển (Vocabulary)

    # Tiền xử lý văn bản: loại bỏ dấu câu và chuyển thành chữ thường
    def clean_text(self, text):
        text = text.lower()  # Chuyển thành chữ thường
        text = text.translate(str.maketrans('', '', string.punctuation))  # Loại bỏ dấu câu
        return text

    # Tokenize văn bản và loại bỏ từ dừng (nếu có)
    def tokenize(self, text, stopwords=None):
        tokens = text.split()
        if stopwords:
            tokens = [word for word in tokens if word not in stopwords]
        return tokens

    # Tiền xử lý văn bản cho toàn bộ tập dữ liệu
    def preprocess_data(self, texts, stopwords=None):
        return [self.tokenize(self.clean_text(text), stopwords) for text in texts]

    # Tính xác suất tiên nghiệm cho các lớp P(C)
    def calculate_class_probs(self, labels):
        class_counts = Counter(labels)
        total_count = len(labels)
        self.class_probs = {class_label: count / total_count for class_label, count in class_counts.items()}

    # Tính xác suất điều kiện P(word | class)
    def calculate_word_probs(self, tokens, labels):
        word_counts_by_class = {label: Counter() for label in set(labels)}
        class_counts = Counter(labels)
        # Đếm tần suất từ trong từng lớp
        for text, label in zip(tokens, labels):
            word_counts_by_class[label].update(text)
        # Tính xác suất điều kiện cho mỗi từ trong mỗi lớp
        word_probs_by_class = {}
        vocab_size = len(self.vocab)  # Số lượng từ trong từ điển (Vocabulary)
        
        for class_label, word_counts in word_counts_by_class.items():
            total_words_in_class = sum(word_counts.values())
            word_probs = {word: (count + self.smoothing) / (total_words_in_class + self.smoothing * vocab_size)  # Laplace smoothing
                          for word, count in word_counts.items()}
            word_probs_by_class[class_label] = word_probs
        
        self.word_probs_by_class = word_probs_by_class

    # Huấn luyện mô hình Naive Bayes
    def fit(self, texts, labels, stopwords=None):
        # Tiền xử lý dữ liệu
        tokens = self.preprocess_data(texts, stopwords)
        # Cập nhật từ điển (vocabulary)
        self.vocab = set([word for text in tokens for word in text])
        # Tính xác suất tiên nghiệm cho các lớp
        self.calculate_class_probs(labels)
        # Tính toán xác suất điều kiện (Likelihood) cho mỗi lớp
        self.calculate_word_probs(tokens, labels)
    # Dự đoán lớp cho một văn bản mới
    def predict(self, text, stopwords=None):
        # Tiền xử lý văn bản mới
        tokens = self.tokenize(self.clean_text(text), stopwords)
        
        # Tính toán xác suất của văn bản cho mỗi lớp
        class_scores = {}
        for class_label, class_prob in self.class_probs.items():
            score = log(class_prob)  # Log của xác suất tiên nghiệm P(C)
            for word in tokens:
                # Nếu từ có trong lớp, cộng log của xác suất điều kiện P(word | class)
                word_prob = self.word_probs_by_class[class_label].get(word, self.smoothing / (sum([sum(word_counts.values()) for word_counts in self.word_probs_by_class.values()]) + len(self.vocab)))  # Nếu từ chưa có trong lớp, dùng Laplace smoothing
                score += log(word_prob)
            class_scores[class_label] = score
        
        # Chọn lớp có xác suất cao nhất
        predicted_class = max(class_scores, key=class_scores.get)
        
        # Tính toán xác suất của lớp dự đoán
        predicted_prob = exp(class_scores[predicted_class])  # Chuyển từ log lại thành xác suất thực tế
        return predicted_class, predicted_prob

    # Dự đoán cho toàn bộ tập dữ liệu
    def predict_all(self, texts, stopwords=None):
        return [self.predict(text, stopwords) for text in texts]

    # Tính toán hàm mất mát (Log Loss / Cross-entropy loss)
    def log_loss(self, X, y, epsilon=1e-10):  # epsilon là giá trị nhỏ để tránh log(0)
        loss = 0.0
        for text, true_label in zip(X, y):
            predicted_label = self.predict(text)
            predicted_prob = self.class_probs.get(predicted_label, 0.0)
            # Đảm bảo predicted_prob không bằng 0 hoặc 1
            predicted_prob = max(min(predicted_prob, 1 - epsilon), epsilon)
            true_prob = 1 if predicted_label == true_label else 0
            # Cross-entropy loss
            loss += -true_prob * math.log(predicted_prob) - (1 - true_prob) * math.log(1 - predicted_prob)
        
        return loss / len(y)

In [7]:
import pandas as pd
# Đọc dữ liệu từ CSV
data = pd.read_csv('D:/NLP_CVS/pj_ML/data/processed_comment.csv')
# Lấy dữ liệu văn bản và nhãn
texts = data['processed_comment'].tolist()
labels = data['label'].tolist()
# Khởi tạo Naive Bayes
naive_bayes = NaiveBayesClassifier(smoothing=1.0)
# Huấn luyện mô hình
naive_bayes.fit(texts, labels)
import joblib
# Lưu mô hình
joblib.dump(naive_bayes, 'D:/NLP_CVS/pj_ML/Naive_Bayes/naive_bayes_model.pkl')

['D:/NLP_CVS/pj_ML/Naive_Bayes/naive_bayes_model.pkl']

In [9]:
# Tải lại mô hình
loaded_model = joblib.load('D:/NLP_CVS/pj_ML/Naive_Bayes/naive_bayes_model.pkl')
# Dự đoán với mô hình đã tải
new_text = "Dịch vụ này rất tốt tôi hài lòng"
predicted_class = loaded_model.predict(new_text)
print(f"Predicted sentiment from loaded model: {predicted_class}")

Predicted sentiment from loaded model: ('Positive', 8.241863839715324e-29)


In [11]:
# Dự đoán cho văn bản mới
new_text = "Dịch vụ này rất tệ"
predicted_class = naive_bayes.predict(new_text)
print(f"Predicted sentiment: {predicted_class}")

Predicted sentiment: ('Negative', 5.513575127388398e-19)


In [13]:
new_text = "Dịch vụ này bình thường tạm chấp nhận được"
predicted_class = naive_bayes.predict(new_text)
print(f"Predicted sentiment: {predicted_class}")

Predicted sentiment: ('Normal', 5.261398867304483e-32)


In [ ]:
new